In [1]:
def tree(label, branches=[]):
    """Construct a tree with the given label value and a list of branches."""
    for branch in branches:
        assert is_tree(branch), 'branches must be trees'
    return [label] + list(branches)

def label(tree):
    """Return the label value of a tree."""
    return tree[0]

def branches(tree):
    """Return the list of branches of the given tree."""
    return tree[1:]

def is_tree(tree):
    """Returns True if the given tree is a tree, and False otherwise."""
    if type(tree) != list or len(tree) < 1:
        return False
    for branch in branches(tree):
        if not is_tree(branch):
            return False
    return True

def is_leaf(tree):
    """Returns True if the given tree's list of branches is empty, and False
    otherwise.
    """
    return not branches(tree)

def print_tree(t, indent=0):
    """Print a representation of this tree in which each node is
    indented by two spaces times its depth from the root.

    >>> print_tree(tree(1))
    1
    >>> print_tree(tree(1, [tree(2)]))
    1
      2
    >>> numbers = tree(1, [tree(2), tree(3, [tree(4), tree(5)]), tree(6, [tree(7)])])
    >>> print_tree(numbers)
    1
      2
      3
        4
        5
      6
        7
    """
    print('  ' * indent + str(label(t)))
    for b in branches(t):
        print_tree(b, indent + 1)

def copy_tree(t):
    """Returns a copy of t. Only for testing purposes.

    >>> t = tree(5)
    >>> copy = copy_tree(t)
    >>> t = tree(6)
    >>> print_tree(copy)
    5
    """
    return tree(label(t), [copy_tree(b) for b in branches(t)])

# Required Questions

## Q1: Acorn Finder

The squirrels on campus need your help! There are a lot of trees on campus and the squirrels would like to know which ones contain acorns. Define the function `acorn_finder`, which takes in a tree and returns `True` if the tree contains a node with the value 'acorn' and `False` otherwise.

## ===== Answer =====

In [ ]:
def acorn_finder(t):
    

1. The base case is if the label of the currently selected tree is 'acorn'

In [ ]:
if label(t) == 'acorn':
    return True

2. The recursive case is that we are going to check the labels for each branches

In [ ]:
for b in branches(t):
    if acorn_finder(b):
        return True
return False

Combining the 2 conditions above,

In [29]:
def acorn_finder(t):
    if label(t) == 'acorn':
        return True
    for b in branches(t):
        if acorn_finder(b):
            return True
    return False

In [6]:
# Doctest for acorn_finder
""" 
Returns True if t contains a node with the value 'acorn', otherwise returns False

>>> scrat = tree('acorn')
>>> acorn_finder(scrat)
True
>>> sproul = tree('roots', [tree('branch1', [tree('leaf'), tree('acorn')]), tree('branch2')])
>>> acorn_finder(sproul)
True
>>> numbers = tree(1, [tree(2), tree(3, [tree(4), tree(5)]), tree(6, [tree(7)])])
>>> acorn_finder(numbers)
False
"""
import doctest
doctest.testmod()

TestResults(failed=0, attempted=14)

Another possible implementation is as the following,

In [5]:
def acorn_finder(t):
    if label(t) == 'acorn':
        return True
    return True in [acorn_finder(b) for b in branches(t)]

The following line,

In [ ]:
return True in [acorn_finder(b) for b in branches(t)]

Solves the following cases:

1. If even in the beginning, we have a leaf that doesn't contain `acorn`, then this will return:

In [ ]:
return True in []

Which is `False`.

2. If we have a proper tree with branches, then we just need one `True` for the whole function to return `True`. Otherwise, it returns `False`. And note that the return statement is outside the `if` clause. This implementation ensures that if the `if` clause is not executed, 

# ====================================================

## Q2: Pruning Leaves

Define a function `prune_leaves` that given a tree $t$ and a tuple of values $vals$, produces a version of $t$ with all its leaves that are in vals removed. Do not attempt to try to remove non-leaf nodes and do not remove leaves that do not match any of the items in vals. Return $None$ if pruning the tree results in there being no nodes left in the tree.

## ==== Answer ====

A very tricky problem. We have a base case that is, if we encounter a `leaf` and the value of that leaf is within `vals`, then return `None`.

In [ ]:
if is_leaf(t) and label(t) in vals:
    return None

The recursive case is the trickiest part. We can't remove a tree. What we can do is to create a new tree that excludes the leaves that were removed. And we need to do the same thing for the branches.

1. Create a list for the branches with the same level

In [ ]:
new_branches = []

2. Iterate through each branch
    * Recursive call prune_leaves
        * We know that if it contains a leaf that's removed. It will return a `None`, which evaluates to `False` value.
        * We want to contain all the branches that doesn't contain the leaves that are in `vals`.

In [ ]:
for b in branches(t):
    if prune_leaves(b, vals):
        new_branches += [prune_leaves(b, vals)]

3. And in the end, we want to create a new tree with the branches that we just constructed.

In [ ]:
return tree(label(t), new_branches)

Combining all of the above,

In [7]:
def prune_leaves(t, vals):
    if is_leaf(t) and label(t) in vals:
        return None
    new_branches = []
    for b in branches(t):
        if prune_leaves(b, vals):
            new_branches += [prune_leaves(b, vals)]
    return tree(label(t), new_branches)

In [6]:
"""
Return a modified copy of t with all leaves that have a label that appears in vals removed.  Return None if the entire tree ispruned away.

    >>> t = tree(2)
    >>> print(prune_leaves(t, (1, 2)))
    None
    >>> numbers = tree(1, [tree(2), tree(3, [tree(4), tree(5)]), tree(6, [tree(7)])])
    >>> print_tree(numbers)
    1
      2
      3
        4
        5
      6
        7
    >>> print_tree(prune_leaves(numbers, (3, 4, 6, 7)))
    1
      2
      3
        5
      6
    """

import doctest
doctest.testmod()

TestResults(failed=0, attempted=13)

# =======================

## Q3. Memory

Write a function that takes in a number $n$ and returns a one-argument function. The returned function takes in a function that is used to update $n$. It prints the updated $n$ value. (Note that this is different from a commentary function from hog since it doesn't return a new function)

## ==== Answer ====

Since this function returns a function, chances are this is a higher-order function. 

It updates `n` within the inner, so we would need to somehow make it so that the argument `n` can be changed during the execution. This can be done by stating in the inner function that `n` is nonlocal.

In [17]:
def memory(n):
    def helper(f):
        nonlocal n
        print(f(n))
        n = f(n)
    return helper

In [18]:
"""
    >>> f = memory(10)
    >>> f(lambda x: x * 2)
    20
    >>> f(lambda x: x - 7)
    13
    >>> f(lambda x: x > 5)
    True
    """

import doctest
doctest.testmod()

TestResults(failed=0, attempted=12)